In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [5]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [6]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [13]:
# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("data\\ravdessdata\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [43]:
test_file2 = os.path.basename('data/ravdessdata/Actor_16/03-01-01-01-01-01-16.wav')
# emotion=emotions[test_file2.split("-")[2]]
# emotion

'neutral'

In [49]:
feature2=extract_feature('data/ravdessdata/Actor_16/03-01-01-01-01-01-16.wav', mfcc=True, chroma=True, mel=True)
feature2

array([-6.48794922e+02,  5.35298462e+01,  4.73926878e+00,  8.01717663e+00,
       -7.10175753e+00, -1.03575964e+01, -8.87435818e+00, -6.93270969e+00,
       -4.27303839e+00, -8.19057655e+00, -3.86202812e+00, -3.55973864e+00,
       -5.88943005e+00, -3.34855223e+00, -6.88626146e+00, -1.58727098e+00,
       -6.67026615e+00, -3.17114139e+00, -6.30953503e+00, -3.45549631e+00,
       -5.60077572e+00, -3.36122036e+00, -4.89138746e+00, -3.75997806e+00,
       -3.63527083e+00, -2.08585286e+00, -1.35647416e+00,  2.27118444e+00,
       -4.47746009e-01,  8.21420789e-01, -4.89234567e-01,  3.36996675e-01,
        9.26720619e-01,  3.03653789e+00,  2.10891604e+00,  2.59847260e+00,
        8.09212446e-01,  2.63657641e+00,  2.33552694e+00,  1.52186954e+00,
        4.91800964e-01,  4.98989403e-01,  5.05500257e-01,  5.28290093e-01,
        5.59649289e-01,  5.70568442e-01,  5.64355910e-01,  5.46481431e-01,
        5.62187374e-01,  5.97419977e-01,  5.64652622e-01,  5.40433407e-01,
        5.06678498e-06,  

In [14]:
# Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [15]:
# Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [18]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [19]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [20]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [86]:
# Predict for the test set
y_pred=model.predict(x_test)
y_pred

array(['happy', 'calm', 'happy', 'happy', 'disgust', 'calm', 'calm',
       'disgust', 'calm', 'fearful', 'happy', 'calm', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'fearful', 'disgust', 'calm', 'calm',
       'disgust', 'disgust', 'calm', 'happy', 'happy', 'fearful', 'happy',
       'fearful', 'fearful', 'happy', 'disgust', 'happy', 'fearful',
       'happy', 'calm', 'calm', 'fearful', 'calm', 'disgust', 'happy',
       'calm', 'calm', 'calm', 'fearful', 'calm', 'disgust', 'calm',
       'calm', 'happy', 'fearful', 'fearful', 'fearful', 'happy', 'happy',
       'fearful', 'calm', 'happy', 'calm', 'calm', 'disgust', 'calm',
       'happy', 'calm', 'happy', 'calm', 'calm', 'disgust', 'fearful',
       'happy', 'fearful', 'fearful', 'fearful', 'fearful', 'fearful',
       'disgust', 'fearful', 'happy', 'calm', 'fearful', 'disgust',
       'calm', 'fearful', 'calm', 'disgust', 'happy', 'disgust',
       'fearful', 'happy', 'fearful', 'disgust', 'fearful', 'calm',
       'happ

In [97]:
pd.DataFrame({'Prediction': y_pred, 'Actual': y_test}).head(60)

,Prediction,Actual
0,happy,happy
1,calm,calm
2,happy,happy
3,happy,happy
4,disgust,disgust
5,calm,calm
6,calm,happy
7,disgust,happy
8,calm,disgust
9,fearful,happy


In [88]:
# model.predict(x_test[:1])

In [80]:
# x_test[0]

In [95]:
#----------------------------#
#    Personal Audio test     #
#----------------------------#
feature22=extract_feature('data/kg.wav', mfcc=True, chroma=True, mel=True)
# feature22
# ValueError: Expected 2D array, got 1D array instead:
# array=[-326.22702026].
# Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
kg_predit = model.predict(feature22.reshape(1, -1))
kg_predit

array(['fearful'], dtype='<U7')

In [96]:
kg_predit[0]

'fearful'

In [22]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.75%
